In [21]:
import os
import json
from PIL import Image, ImageDraw, ExifTags

# Dataset from: https://github.com/GoodNotes/GNHK-dataset
class GNHKdataset():
    def __init__(self, dataset_dir):
        self.dataset_dir = dataset_dir
        self.image_paths = []
        self.annotation_paths = []
        self._load_data()

    def _load_data(self):
        # Iterate through files in the dataset directory
        for file_name in os.listdir(self.dataset_dir):
            if file_name.endswith('.jpg'):
                # Found an image file, add to image_paths
                self.image_paths.append(os.path.join(self.dataset_dir, file_name))
                
                # Corresponding annotation file
                annotation_name = file_name.replace('.jpg', '.json')
                annotation_path = os.path.join(self.dataset_dir, annotation_name)
                self.annotation_paths.append(annotation_path)

    def display_image(self, image_id):
        image_path = self.image_paths[image_id]
        annotation_path = self.annotation_paths[image_id]

        # Load image
        image = Image.open(image_path)
        exif = image._getexif()
        if exif is not None:
            orientation = next((k for k, v in ExifTags.TAGS.items() if v == 'Orientation'), None)
            if orientation in exif:
                if exif[orientation] == 3:
                    image = image.rotate(180, expand=True)
                elif exif[orientation] == 6:
                    image = image.rotate(270, expand=True)
                elif exif[orientation] == 8:
                    image = image.rotate(90, expand=True)
        draw = ImageDraw.Draw(image)

        # Load annotations from JSON
        with open(annotation_path, 'r') as f:
            annotations = json.load(f)

        # Draw bounding boxes
        for annotation in annotations:
            polygon = annotation['polygon']
            x_coordinates = [polygon['x0'],polygon['x1'],polygon['x2'],polygon['x3']]
            y_coordinates = [polygon['y0'],polygon['y1'],polygon['y2'],polygon['y3']]
            # Calculate minimum and maximum coordinates
            xmin = min(x_coordinates)
            ymin = min(y_coordinates)
            xmax = max(x_coordinates)
            ymax = max(y_coordinates)
            draw.rectangle([xmin, ymin, xmax, ymax], outline="red")

        # Show image
        image.show()

In [31]:
# Example usage:
dataset = GNHKdataset(dataset_dir=r'C:\Users\IsaacStalley\Documents\GitHub\GNHKdataset')
dataset.display_image(21)  # Display first image with annotations